In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from numpy.polynomial import Polynomial
import sympy
from functools import reduce
from scipy.spatial.distance import cosine
from tqdm import tqdm
import helpers
import stopit
import sys
import matplotlib.pyplot as plt

In [ ]:
def evaluate(max_vars, max_terms, max_symbols_per_term, max_coeff, max_offset, steps, num_polynomials, verbose=True):
    # Repeatedly generate random polynomials, evaluate each strategy and return the win counts of each strategy
    results = {"grad": [], "0": [], "random": []}
    polynomials = []
    tracker = tqdm if verbose else lambda x: x
    for i in tracker(range(num_polynomials)):
        poly, symbols = helpers.random_sos_polynomial(
            max_vars=max_vars,
            max_terms=max_terms,
            numeric_type=numeric_type,
            max_symbols_per_term=max_symbols_per_term,
            max_coeff=max_coeff,
            max_offset=max_offset)
        polynomials.append(poly)

        try:
            with stopit.ThreadingTimeout(5.0) as timeout_ctx:
                grad_result = helpers.gradient_descent(
                    poly,
                    start=start_fn(poly),
                    steps=steps,
                    min_value=-1e50,
                    verbosity=None,
                    reduce_fn=reduce_fn)[1]
                random_result = helpers.random_search(poly, numeric_type=numeric_type, steps=steps, scale=10)[1]
                results["grad"].append(float(grad_result))
                results["random"].append(float(random_result))
                results["0"].append(poly.eval({x: 0 for x in poly.gens}))
        except stopit.TimeoutException as e:
            print(e)
            continue
            
    # Return the map from strategy name to win count
    return results, polynomials



num_iterations = 100
max_vars = 10
max_terms = 10
max_coeff = 10
max_offset = 1000

numeric_type = helpers.NumericType.int
reduce_fn = helpers.size_one_steps
start_fn = lambda poly: helpers.sample_gen_vals(poly, scale=1, numeric_type=numeric_type)

grad_counts = []
random_counts = []
for steps in [5, 10, 50, 100]:
    for i in tqdm(range(num_iterations)):
        results, polynomials = evaluate(
            max_vars=max_vars,
            max_terms=max_terms,
            max_symbols_per_term=2,
            max_coeff=max_coeff,
            max_offset=max_offset,
            steps=steps,
            num_polynomials=10,
            verbose=False)

        aggregate_results = {
            k: sum([results[k][i] == min([results[kk][i] for kk in results]) for i in range(len(results[k]))])
            for k in results
        }
        grad_counts.append(aggregate_results["grad"])
        random_counts.append(aggregate_results["random"])

    print("Grad:", steps, np.mean(grad_counts), np.std(grad_counts) / np.sqrt(num_iterations))
    print("Random:", steps, np.mean(random_counts), np.std(random_counts) / np.sqrt(num_iterations))

  0%|          | 0/100 [00:00<?, ?it/s]

Grad: 5 7.4 0.14212670403551894
Random: 5 2.6 0.14282856857085702


  0%|          | 0/100 [00:00<?, ?it/s]

Grad: 10 7.64 0.13821722034536796
Random: 10 2.445 0.1377307155285269


  2%|▏         | 2/100 [00:02<02:06,  1.29s/it]